In [ ]:
import math
import torch
import torch.nn as nn
from torch.autograd import Variable #带有自动求导的变量封装
import torch.nn.functional as F

In [ ]:
class Embeddings(nn.Module):
    '''d_model是词嵌入的维度，vocab是词表的大小'''
    def __init__(self,d_model,vocab):
        super(Embeddings,self).__init__()
        self.lut=nn.Embedding(vocab,d_model)
        self.d_model=d_model
    def forward(self,x):
        return self.lut(x)*math.sqrt(self.d_model)#根号维度的作用：缩放

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,d_model,dropout,max_len=5000):
        super(PositionalEncoding,self).__init__()
        self.dropout=nn.Dropout(p=dropout)
        #初始化一个位置编码，内容是0，矩阵的大小是max_len * d_model [5000,512]
        pe=torch.zeros(max_len,d_model)
        #初始化一个绝对位置矩阵,绝对位置其实就是用它的索引去表示，unsqueeze成一列 [5000,1]
        position=torch.arange(0,max_len).unsqueeze(1)
        #之后考虑如何将这些位置信息加入到位置编码矩阵中
        #即,我们需要把max_len*1 转化成 max_len*d_model的形式，这样的话，我们需要给max_len*1的矩阵多乘以一个1*d_model的把一维扩张成d_model维的矩阵
        #同时还希望它能够将自然数的绝对位置编码缩放成足够小的数字，有助于之后梯度下降时的快速收敛
        #定义一个变化矩阵div_term，跳跃式的变化,初始化两个，用于分别sin/cos跳跃式的计数
        div_term=torch.exp(torch.arange(0,d_model,2)*-(math.log(10000)/d_model)) #[256]的list
        pe[:,0::2]=torch.sin(position*div_term)#[5000,256]
        pe[:,1::2]=torch.cos(position*div_term)#[5000,256]. 总共是[5000,512]
        #pe现在还只是是一个二维矩阵，需要unsqueeze拓展一个维度
        pe=pe.unsqueeze(0) #[1,5000,512]
        #最后一步需要把pe的位置编码注册成模型的buffer
        #什么是buffer：不需要根据training进行更新，没有参数和超参数
        self.register_buffer('pe',pe)
    def forward(self,x):
        x=x+Variable(self.pe[:,:x.size(1)],requires_grad=False)#只提取x对应长度既可，不需要5000
        return self.dropout(x)
        #这里为什么要对带有位置编码的x dropout
        #这里的dropout可以看成是对于数据进行一种添加噪声的方法，增加模型的鲁棒性。

In [ ]:
def subsequent_mask(size):
    #生成掩码张量
    attn_shape=(1,size,size)
    #用torch.one的方法生成值为1的上三角矩阵，为什么不是全矩阵：节省空间
    #剩下没有被定义的元素位置，给予无符号8位整形 uint8
    #为什么做成上三角：每次遍历一行，每次依次往后增加一个mask
    subsequent_mask=np.triu(np.ones(attn_shape),k=1).astype('uint8')
    #最后把torch转化为tensor，内部做一个1- 的操作，实际是在做一个矩阵反转，现在是一个元素为1的下三角矩阵
    return torch.from_numpy(1-subequent_mask)

In [ ]:
#attention(Q,K,V)=softmax((Q*K)/(sqrt(dk)))*V
#query就是整个文本的信息
#key:为了让模型更好的理解文本，给出的关键提示就是key
#value就是对应的权重，对应query的权重，权重高的就是这段文本所对应的答案
#初始化的时候，value不知道哪些是重点，所以v和k的矩阵一样
#自注意力机制：一种特殊情况，QKV三个矩阵一样
def attention(query,key,value,mask=None,dropout=None):
    d_k=query.size(-1)#首先取query最后一维的大小，当作词嵌入的维度,之后用于缩放
    scores=torch.matmul(query,key.transpose(-2,-1))/math.sqrt(d_k)#key中的二维矩阵要转置
    if mask!=None:#如果有掩码张量
        scores=scores.mask_fill(mask==0,-1e9)
    p_attn=F.softmax(scores,dim=-1)
    #判断有没有dropout
    if dropout is not None:
        p_attn=dropout(p_attn)
    return torch.matmul(p_attn,value),p_attn

In [ ]:
import copy#用于深度拷贝
def clones(module,n):#n代表要克隆的数量
    return nn.ModuleList([copy.deepcopy(module) for _ in range(n)])

#多头的意义：一个词被映射成512个维度，这么多维度不可能用一个head处理，所以应该用多个头来处理这些维度
class MultiHeadedAttention(nn.Module):
    def __init__(self,head,embedding_dim,dropout=0.1):#embedding_dim是词嵌入的维度,512。不是有多少个词
        super(MultiHeadedAttention,self).__init__()
        assert embedding_dim%head==0
        self.d_k=embedding_dim//head#获得每个头获得的分割词向量维度d_k
        self.head=head
        self.embedding_dim=embedding_dim
        #feature_in是embedding_dim, out是embedding
        self.linears=clones(nn.Linear(embedding_dim,embedding_dim),4)#QKV分别需要一个linear，最后拼接也需要一个
        self.attn=None
        self.dropout=nn.Dropout(p=dropout)

    def forward(self,query,key,value,mask=None):
        if mask is not None:
            mask=mask.unsqueeze(1)#增加一个维度，方便形成batch
        batch_size=query.size(0)
        #用zip把QKV三个矩阵组合到一起
        query,key,value=\
        [model(x).view(batch_size,-1,self.head,self.d_k).transpose(1,2)#转置2，3这两个维度
        for model,x in zip(self.linears,(query,key,value))]#对应的model加载对应的data
        #得到每个头的输入之后我们就可以把他们传入到attention里了
        x,self.attn=attention(query,key,value,mask=None,dropout=self.dropout)
        x=x.transpose(1,2).contiguous().view(batch_size,-1,self.head*self.d_k)#如果想同时用transpose和view，需要用contiguous
        return self.linears[-1](x)

In [ ]:
#前馈全连接层
class PositionwiseFeedForward(nn.Module):
    def __init__(self,d_model,d_ff,dropout=0.1):#d_model是第一个线性层输入的维度
        super(PositionwiseFeedForward,self).__init__()#d_ff隐藏
        self.w1=nn.Linear(d_model,d_ff)
        self.w2=nn.Linear(d_ff,d_model)
        self.dropout=nn.Dropout(dropout)
    def forward(self,x):
        return self.w2(self.dropout(F.relu(self.w1(x))))#实现两层的全连接

In [ ]:
#规范化层
class LayerNorm(nn.Module):
    def __init__(self,features,eps=1e-6):
        super(LayerNorm,self).__init__()
        self.a2=nn.Parameter(torch.ones(features))#定义两个辅助张量
        self.b2=nn.Parameter(torch.zeros(features))#Parameter进行封装
        self.eps=eps
    def forward(self,x):
        mean=x.mean(-1,keepdim=True)#求最后一个维度的mean
        std=x.std(-1,keepdim=True)
        return self.a2*(x-mean)/(std+self.eps)+self.b2#b2这里就是shift，*这里是对应位置相乘

In [ ]:
#add层
class SublayerConnection(nn.Module):
    def __init__(self,size,dropout=0.1):#size是词嵌入维度大小
        super(SublayerConnection,self).__init__()
        self.norm=LayerNorm(size)
        self.dropout=nn.Dropout(p=dropout)
    def forward(self,x,sublayer):
        return x+self.dropout(sublayer(self.norm(x)))

In [ ]:
#汇总：编码器层
class EncoderLayer(nn.Module):
    def __init__(self,size,self_attn,feed_forward,dropout):#这里feed_forward是一个维度
        super(EncoderLayer,self).__init__()
        self.self_attn=self_attn
        self.feed_forward=feed_forward
        self.sublayer=clones(SublayerConnection(size,dropout),2)#我们需要两个子连接层
        self.size=size

    def forward(self,x,mask):
        x=self.sublayer[0](x,lambda x:self.self_attn(x,x,x,mask))
        return self.sublayer[1](x,self.feed_forward)

In [ ]:
#编码器
class Encoder(nn.Module):
    def __init__(self,layer,N):#初始化编码器层和层数
        super(Encoder,self).__init__()
        self.layers=clones(layer,N)#先定义好n个layer
        self.norm=LayerNorm(layer.size)#512
        
    def forward(self,x,mask):
        for layer in self.layers:
            x=layer(x,mask)#每层的输入是x和mask，然后迭代输出，定义每次新的输入为x
        return self.norm(x)

In [ ]:
#流程模拟
d_model=512
dropout=0.1
max_len=60
vocab=1000
#词嵌入演示
embedding=Embeddings(512,1000)#这里的10参考的是下面一行数值的取值范围
x=Variable(torch.LongTensor([[100,2,421,508],[491,998,1,221]]))
print('input shape:',x.shape,' || And embedding dimension:', d_model)
embr=embedding(x)
print(embr.shape)
#print(embr)

In [ ]:
#位置编码演示
x=embr
pe=PositionalEncoding(d_model=512,dropout=0.1,max_len=60)
pe_result=pe(embr)
print(pe_result.shape)
#print(pe_result)

In [ ]:
#mask
x=Variable(torch.randn(5,5))
print(x)
mask=Variable(torch.zeros(5,5))
print(mask)
y=x.masked_fill(mask==0,-1e9)
print(y)

In [ ]:
#attention
query=key=value=pe_result
attn,p_attn=attention(query,key,value)
#print('attn:',attn)
print('attn.shape:',attn.shape)
#print('p_attn:',p_attn)

In [ ]:
#多头
head=8
embedding_dim=512
dropout=0.2
mask=Variable(torch.zeros(8,4,4))
multi_head=MultiHeadedAttention(head=8,embedding_dim=512,dropout=0.2)
multi_head_result=multi_head(query,key,value,mask)
#print(multi_head_result)
print(multi_head_result.shape)

In [ ]:
#全连接层
d_model=512
d_ff=64
dropout=0.2
x=multi_head_result
fc=PositionwiseFeedForward(d_model=512,d_ff=64,dropout=0.2)
fc_result=fc(x)
#print(fc_result)
print(fc_result.shape)

In [ ]:
#正则化
feature=d_model=512
eps=1e-6
x=fc_result
layer_norm=LayerNorm(features=512,eps=1e-6)
layer_norm_result=layer_norm(x)
#print(layer_norm_result)
print(layer_norm_result.shape)

In [ ]:
#resnet
d_model=512
x=pe_result
mask=Variable(torch.zeros(8,4,4))#因为head是8个所以这里mask也需要8个
self_attn=MultiHeadedAttention(head=8,embedding_dim=512,dropout=0.2)#head=8,embedding_dim=512,dropout=0.2
sublayer=lambda x: self_attn(x,x,x,mask)
sc=SublayerConnection(size=512,dropout=0.2)
sc_result=sc(x,sublayer)
#print(sc_result)
print(sc_result.shape)

In [ ]:
#Encoder_Layer
d_ff=64
d_model=512
size=512
ff=PositionwiseFeedForward(d_model,d_ff,dropout=0.2) 
self_attn=MultiHeadedAttention(head=8,embedding_dim=512,dropout=0.2)#head=8,embedding_dim=512,dropout=0.2
encoder=EncoderLayer(size,self_attn,ff,dropout)#size,self_attn,feed_forward,dropout
encoder_result=encoder(x,mask)
#print(encoder_result)
print(encoder_result.shape)

In [ ]:
#Encoder
c=copy.deepcopy
attn=MultiHeadedAttention(head=8,embedding_dim=512,dropout=0.2)
ff=PositionwiseFeedForward(d_model,d_ff,dropout)
layer=EncoderLayer(size,c(attn),c(ff),dropout)
N=8
ENCODER=Encoder(layer,N)
EN_result=ENCODER(x,mask)
#print(EN_result)
print(EN_result.shape)

In [ ]:
#解码器层
class DecoderLayer(nn.Module):
    def __init__(self,size,self_attn,src_attn,feed_forward,dropout):
        #size是词嵌入维度大小，同时也代表解码器层的尺寸；self_attn: Q=K=V；src_attn: Q!=K=V
        super(DecoderLayer,self).__init__()
        self.size=size
        self.self_attn=self_attn
        self.src_attn=src_attn
        self.feed_forward=feed_forward
        self.sublayer=clones(SublayerConnection(size,dropout),3)#三层对应output的多头，加上encoder的多头和全连接层

    def forward(self,x,memory,source_mask,target_mask):
        #memory是来自编码层的语义存储的变量，以及元数据掩码张量和目标数据的掩码张量
        m=memory
        #第一层，这里的x不是encoder里面的x（encoder里面的x是现在的memory），这里的x是对应的是数据集中的结果y
        #这里的mask才开始有用，之前encoder的mask是全零矩阵

        x=self.sublayer[0](x,lambda x: self.self_attn(x,x,x,target_mask))
        #第二层，src_attn，这里的Q我们取x，K，V；source_mask
        x=self.sublayer[1](x,lambda x: self.src_attn(x,m,m,source_mask))
        #第三层
        return self.sublayer[2](x,self.feed_forward)

In [ ]:
#解码器
class Decoder(nn.Module):
    def __init__(self,layer,N):
        super(Decoder,self).__init__()
        self.layers=clones(layer,N)
        self.norm=LayerNorm(layer.size)

    def forward(self,x,memory,source_mask,target_mask):
        for layer in self.layers:
            x=layer(x,memory,source_mask,target_mask)
        return self.norm(x)

In [ ]:
#输出部分
import torch.nn.functional as F
class Generator(nn.Module):
    def __init__(self,d_model,voacb_size):
        super(Generator,self).__init__()
        self.project=nn.Linear(d_model,voacb_size)

    def forward(self,x):
        return F.log_softmax(self.project(x),dim=-1)

In [ ]:
#终极整体
class EncoderDecoder(nn.Module):
    def __init__(self,encoder,decoder,source_embed,target_embed,generator):
        super(EncoderDecoder,self).__init__()
        self.encoder=encoder#编码对象
        self.decoder=decoder#解码对象
        self.src_embed=source_embed#原数据的embedding函数
        self.tgt_embed=target_embed#目标数据embedding函数
        self.generator=generator
    def forward(self,source,target,source_mask,target_mask):
        #source是原数据,target是目标数据
        return self.decode(self.encode(source,source_mask),source_mask,target,target_mask)
    def encode(self,source,source_mask):
        return self.encoder(self.src_embed(source),source_mask)#src_embed就是词嵌入
    def decode(self,memory,source_mask,target,target_mask):
        return self.decoder(self.tgt_embed(target),memory,source_mask,target_mask)


In [ ]:
#decoder_layer
size=d_model=512
head=8
d_ff=64
dropout=0.2
x=pe_result
self_attn=src_attn=MultiHeadedAttention(head,d_model,dropout)
ff=PositionwiseFeedForward(d_model,d_ff,dropout)
memory=EN_result
mask=Variable(torch.zeros(8,4,4))
source_mask=target_mask=mask

decode_layer=DecoderLayer(size,self_attn,src_attn,ff,dropout)
decode_layer_result=decode_layer(x,memory,source_mask,target_mask)
print(decode_layer_result.shape)

In [ ]:
#decoder
layer=DecoderLayer(d_model,c(attn),c(attn),c(ff),dropout)
decoder=Decoder(layer,8)
decoder_result=decoder(x,memory,source_mask,target_mask)
print(decoder_result)
print(decoder_result.shape)

In [ ]:
#final
d_model=512
vocab_size=1000
x=decode_layer_result
generator=Generator(d_model,vocab_size)
generator_result=generator(x)
print(generator_result)
print(generator_result.shape)

In [ ]:
#enco-deco
vocab_size=1000
d_model=512
encoder=ENCODER
decoder=decoder
source_embed=nn.Embedding(vocab_size,d_model)
target_embed=nn.Embedding(vocab_size,d_model)

source=target=Variable(torch.LongTensor([[100,2,421,508],[491,998,1,221]]))
source_mask=target_mask=Variable(torch.zeros(8,4,4))
gen=generator
ED=EncoderDecoder(encoder,decoder,source_embed,target_embed,gen)
ed_result=ED(source,target,source_mask,target_mask)
print(ed_result)
print(ed_result.shape)

In [ ]:
def make_model(source_vocab,target_vocab,N=6,d_model=512,d_ff=2048,head=8,dropout=0.1):
    c=copy.deepcopy
    attn=MultiHeadedAttention(head,d_model)
    ff=PositionwiseFeedForward(d_model,d_ff,dropout)
    position=PositionalEncoding(d_model,dropout)
    model=EncoderDecoder(
        Encoder(EncoderLayer(d_model,c(attn),c(ff),dropout),N),
        Decoder(DecoderLayer(d_model,c(attn),c(attn),c(ff),dropout),N),
        nn.Sequential(Embeddings(d_model,source_vocab),c(position)),
        nn.Sequential(Embeddings(d_model,target_vocab),c(position)),
        Generator(d_model,target_vocab)
        )
    for p in model.parameters():#初始化模型的参数，初始化城一个均匀分布的矩阵
        if p.dim()>1:
            nn.init.xavier_uniform(p)
    return model

if __name__ =='__main__':
    res=make_model(source_vocab=11,target_vocab=11,N=6)
    print(res)

In [ ]:
import numpy as np

def data_generator(V,batch,num_batch):
    # 该函数用于随机生成copy函数的数据，它的三个输入参数是 V随机生成数字的最大值+1
    #batch:每次输送给模型更新一次参数的数据量，Num_batch:一共输送模型多少轮数据

    #使用for循环变量n batches
    for i in range(num_batch):
        #在循环中使用np的random.randint方法随机生成【1，v]的整数
        #分布在（batch,10)形状的矩阵中，然后再把numpy形式转换成torch的tensor
        data =torch.from_numpy(np.random.randint(1,V,size=(batch,10)))

        #接着使矩阵的第一列数字都为1，这一列也就成为了其实标志列
        #当解码器进行第一次解码时，会使用起始标志列作为输入
        data[:,0]=1

        #因为是copy任务，所有source与target是完全相同的，且数据样本作用变量不需要梯度
        # 因此requires_grad设置为false
        source=Variable(data,requires_grad=False)
        target=Variable(data,requires_grad=False)

        # 使用batch对source和target进行对应批次的掩码张量生成，最后使用yield返回
        yield Batch(source,target)

In [ ]:
#定义模型优化器和损失函数 
!pip install pyitcast
#导入模型单轮训练工具包run_epoch,该工具将对模型使用给定的损失函数计算方法进行单轮参数更新
from pyitcast.transformer_utils import run_epoch
#导入工具包batch,它能够对原始样本数据生成对应批次的掩码张量
from pyitcast.transformer_utils import Batch
#导入工具包 get_std_opt,该工具获得标准优化器，使其对序列到序列的任务更有效
from pyitcast.transformer_utils import get_std_opt
#导入标签平滑工具包，小幅度的改变原有标签的值域，可以防止过拟合(人工标注的数据也可能因为外界影响而产生偏差)
from pyitcast.transformer_utils import LabelSmoothing
#导入计算损失包，该工具能够使用标签平滑后的结果进行损失的计算，损失的计算方法是交叉熵损失函数
from pyitcast.transformer_utils import SimpleLossCompute

V=11
batch_size=20
num_batch=30
model=make_model(V,V,N=2)
model_optimizer=get_std_opt(model)#优化器
criterion=LabelSmoothing(size=V,padding_idx=0,smoothing=0.0)
loss=SimpleLossCompute(model.generator,criterion,model_optimizer)#loss

In [ ]:
from pyitcast.transformer_utils import greedy_decode#对最终结果进行贪婪解码,每次预测都选择最大概率进行输出
def run(model,loss,epochs=10):
    for epoch in range(epochs):
        model.train()
        run_epoch(data_generator(V,8,20),model,loss)
        model.eval()
        run_epoch(data_generator(V,8,5),model,loss)
    model.eval()
    source=Variable(torch.LongTensor([[1,3,2,5,4,6,7,8,9,10]]))
    source_mask=Variable(torch.ones(1,1,10))
    result=greedy_decode(model,source,source_mask,max_len=10,start_symbol=1)
    print(result)

In [ ]:
if __name__ =='__main__':
    run(model,loss)